In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlite3
import requests
from bs4 import BeautifulSoup

import unidecode

_**sofifia time ids**:_   
    18-1-28.08.17: 158835   
    17-2-22.02.17: 158647   
    17-1-25.08.16: 158466   
    16-2-19.02.16: 158278   
    16-1-28.08.15: 158103   
    15-2-20.02.15: 157914   
    15-1-29.08.14: 157739   
    14-2-21.02.14: 157550   
    14-1-31.08.13: 157376   
    13-2-22.02.13: 157186   
    13-1-31.08.12: 157011   
    12-2-22.02.12: 156820   
    12-1-30.08.11: 156644   
    11-2-22.02.11: 156455   
    11-1-30.08.10: 156279   
    10-2-22.02.10: 156090   
    10-1-30.08.09: 155914   
    09-2-22.02.09: 155725   
    09-1-30.08.08: 155549   
        
**_sofifaLigaIds_** = {    
    "1. Bundesliga": 19,   
    *"2. Bundesliga": 20,*   
    *"Premier League": 13,*   
    *"Championship": 14,*   
    "La Liga": 53,   
    "Serie A": 31,   
    *"League 1": 16,*   
    *"Eredevise":10,*   
    "Super Lig": 68,   
    *"Brasileiro": 7,*    
    "MLS": 39,   
    "Russian Primier League": 67,   
    "Allsvenskan": 56}    
        


In [10]:
#erst ab fifa 10 team statistiken vorhanden
patches = [158835,158647,158466,158278,158103,157914,157739,157550,157376,157186,157011,156820,156644,156455,156279,156090,155914]

In [76]:
con.close()

In [2]:
con = sqlite3.connect('soccer.sqlite')
c = con.cursor()

In [19]:
for i in [19,20,13,14,53,31,16,10,68,7,39,67,56,60,61]:
    print(str(i).ljust(2), str([x[0] for x in c.execute('SELECT count(*) FROM team WHERE ligaID IS '+str(i))][0]).ljust(5), str([x[0] for x in c.execute('SELECT count(*) FROM player WHERE teamID IN (SELECT teamID FROM team WHERE ligaID IS '+str(i)+')')][0]).ljust(5))

19 306   9113 
20 306   8389 
13 340   11255
14 408   11836
53 340   10565
31 340   10133
16 340   10060
10 306   8375 
68 203   5521 
7  0     0    
39 0     0    
67 0     0    
56 272   6739 
60 408   10989
61 408   10504


In [13]:
[x for x in c.execute('select match.ligaID, league.name, count(*) from league join match on match.ligaID = league.leagueID group by match.ligaID')]

[(10, 'Eredevise', 800),
 (13, 'Premier League', 3033),
 (14, 'English Championship', 3978),
 (16, 'Ligue 1', 2514),
 (19, 'Bundesliga', 2387),
 (20, '2. Bundesliga', 2194),
 (31, 'Serie A', 2726),
 (53, 'La Liga', 3040),
 (68, 'Super Lig', 850)]

In [4]:
print([x[0] for x in c.execute('select count(*) from match')])
[x for x in c.execute('select ligaID,count(*) from match group by ligaID')]

[30365]


[(10, 2195),
 (13, 3033),
 (14, 3978),
 (16, 2514),
 (19, 2387),
 (20, 2194),
 (31, 2726),
 (53, 3040),
 (60, 4401),
 (61, 3047),
 (68, 850)]

In [18]:
patch_urls = {158835 : "/18/158835",    
158647 : "/17/158647",    
158466 : "/17/158466",    
158278 : "/16/158278",    
158103 : "/16/158103",    
157914 : "/15/157914",    
157739 : "/15/157739",    
157550 : "/14/157550",    
157376 : "/14/157376",    
157186 : "/13/157186",    
157011 : "/13/157011",    
156820 : "/12/156820",    
156644 : "/12/156644",    
156455 : "/11/156455",    
156279 : "/11/156279",    
156090 : "/10/156090",    
155914 : "/10/155914",    
155725 : "/09/155725",    
155549 : "/09/155549"}

attr_base = ['Crossing',
 'Finishing',
 'Heading accuracy',
 'Short passing',
 'Volleys',
 'Dribbling',
 'Curve',
 'Free kick accuracy',
 'Long passing',
 'Ball control',
 'Acceleration',
 'Sprint speed',
 'Agility',
 'Reactions',
 'Balance',
 'Shot power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long shots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'Standing tackle',
 'Sliding tackle',
 'GK diving',
 'GK handling',
 'GK kicking',
 'GK positioning',
 'GK reflexes']

ligaID = 61
for i,patch in enumerate(patches):
    patch_url = patch_urls[patch]
#     league_url = https://sofifa.com/league/19?v=14&e=157376&set=true
    
    url_base = ''.join(['https://sofifa.com/league/',str(ligaID),'&v=', patch_url.split('/')[1],'&e=',str(patch) , '&set=true'])
    r = requests.get(url_base)
    if len(r.history)>0:
        continue
    soup = BeautifulSoup(r.content, 'html.parser')

    teams_urls = []
    for tr in soup.find_all('tr')[1:]:
        teams_urls.append(tr.find_next('a')['href'])

    for url in teams_urls:
        sofifaTeamID = url.split('/')[-1]
        url_team = ''.join(['https://sofifa.com', url, str(patch_url)])
        print(url_team)
#         row = [''.join([sofifaid, patch]), ]
        
        rt = requests.get(url_team)
        if len(rt.history)>0:
            continue
        team_soup = BeautifulSoup(rt.content, 'html.parser')

        name =unidecode.unidecode(' '.join(team_soup.title.contents[0].split(' ')[:-6]) )
        teamID = ''.join([str(sofifaTeamID), str(patch)])
        attr = team_soup.find_all('dl')[0]
        attributes = []
        lis = attr.find_all('span')
        for j,el in enumerate(lis):
            if j==3 or j == 7 or j==11:
                continue
            attributes.append(el.text.split(' ')[0])
        c.execute('INSERT OR IGNORE INTO team VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?)', (teamID, ligaID, sofifaTeamID, patch, name, *attributes))
        con.commit()


        player_urls = []
        for tr in team_soup.find_all('tbody')[0].find_all('tr'):
            player_urls.append(tr.find_all('a')[1]['href'])

        for player_url in player_urls:
            rp = requests.get(''.join(['https://sofifa.com', player_url, str(patch_url)]))
            if len(rp.history)>0:
                continue
            player_soup = BeautifulSoup(rp.content,'lxml')
            meta = []
            attr_dict = dict.fromkeys(attr_base, 0)
            things  = player_soup.find_all('ul', class_='pl')
            meta.append((unidecode.unidecode(' '.join(player_soup.title.contents[0].split(' ')[:-6]))))
            metadata  = player_soup.find_all('div', class_='meta')
            for l in metadata:
                lis = l.find_all('li')
                lt=l.text.split(' ')
                for m in [lt[-6],lt[-2][:-2],lt[-1][:-3]]:
                    meta.append(m)
            #attributes
            if things[2].find_next('li').text.split(' ')[-1].strip()=='Crossing':
                m1 = 2
                m2 = 9
            else:
                m1 = 3
                m2 = 10
            for ip,l in enumerate(things[m1:m2]):
                lis = l.find_all('li')
                for j,pel in enumerate(lis):
                    attr_name = pel.findAll(text=True, recursive=False)[1].strip()
                    attr_dict[attr_name] = pel.text.split(' ')[0].strip()
            
            meta += [attr_dict[v] for v in attr_base]
            meta.append( things[1].find_all('li')[3].text.strip().split('\n')[1])
            sofifaPlayerID = player_url.split('/')[2]
            c.execute('INSERT OR IGNORE INTO player VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', (''.join([sofifaPlayerID, str(patch)]), sofifaPlayerID, teamID, *meta))
            con.commit()
        print('team_done', name)
    print('patch done')
print('league done', ligaID)



https://sofifa.com/team/92/18/158835
team_done Grimsby Town
https://sofifa.com/team/121/18/158835
team_done Crewe Alexandra
https://sofifa.com/team/135/18/158835
team_done Barnet
https://sofifa.com/team/143/18/158835
team_done Exeter City
https://sofifa.com/team/149/18/158835
team_done Lincoln City
https://sofifa.com/team/346/18/158835
team_done Yeovil Town
https://sofifa.com/team/357/18/158835
team_done Morecambe
https://sofifa.com/team/361/18/158835
team_done Stevenage
https://sofifa.com/team/561/18/158835
team_done Forest Green
https://sofifa.com/team/1480/18/158835
team_done Carlisle United
https://sofifa.com/team/1800/18/158835
team_done Coventry City
https://sofifa.com/team/1923/18/158835
team_done Luton Town
https://sofifa.com/team/1924/18/158835
team_done Chesterfield
https://sofifa.com/team/1928/18/158835
team_done Port Vale
https://sofifa.com/team/1933/18/158835
team_done Wycombe Wanderers
https://sofifa.com/team/1934/18/158835
team_done Swindon Town
https://sofifa.com/team/1

In [7]:
c.execute('SELECT * from team')
[desc[0] for desc in c.description]

['playerID',
 'sofifaPlayerID',
 'teamID',
 'name',
 'age',
 'groesse',
 'gewicht',
 'flanken',
 'abschluss',
 'kopfballPrazision',
 'kurzpasse',
 'volleys',
 'dribbling',
 'effet',
 'freistossPrazision',
 'langePasse',
 'ballkontrolle',
 'beschleunigung',
 'sprintgeschwindigkeit',
 'beweglichkeit',
 'reaktionen',
 'balance',
 'schusskraft',
 'springkraft',
 'ausdauer',
 'starke',
 'fernschusse',
 'aggressivitat',
 'abfangen',
 'stellungsspiel',
 'ubersicht',
 'elfmeter',
 'ruhe',
 'manndeckung',
 'faireZweikampfe',
 'gratsche',
 'twFlugparaden',
 'twFangsicherheit',
 'twAbschlag',
 'twStellungsspiel',
 'twReflexe',
 'jersey']

In [9]:
things[1].find_all('li')[3].text.strip().split('\n')[1]

'24'

In [10]:
meta

['Marius Muller',
 '23',
 '192',
 '89',
 '11',
 '10',
 '12',
 '22',
 '9',
 '10',
 '11',
 '13',
 '16',
 '20',
 '25',
 '38',
 '34',
 '67',
 '38',
 '23',
 '66',
 '35',
 '77',
 '12',
 '27',
 '14',
 '11',
 '48',
 '18',
 '26',
 '9',
 '13',
 '11',
 '69',
 '67',
 '64',
 '64',
 '71',
 '2',
 '4']

In [17]:
plyrs = [plyr for plyr in c.execute('SELECT playerID, sofifaID FROM player')]
c.execute('INSERT INTO player(jersey) VALUES (?)', get_jersey(plyr))

['Manuel Neuer',
 '27,',
 '193',
 '92',
 '15',
 '13',
 '25',
 '48',
 '11',
 '16',
 '14',
 '11',
 '47',
 '31',
 '58',
 '61',
 '43',
 '87',
 '35',
 '25',
 '78',
 '44',
 '83',
 '16',
 '29',
 '30',
 '12',
 '70',
 '37',
 '10',
 '10',
 '11',
 '85',
 '87',
 '91',
 '90',
 '87']

In [10]:
things[2].find_next('li').text.split(' ')

['\nRepublik', 'Irland']

In [18]:
teams = [1156090,2156090]
[x for x in c.execute('SELECT name,playerID FROM player WHERE teamID IN ('+str(teams[0])+','+str(teams[1])+ ')')]


('Neil Taylor', 173735156090)

In [20]:
ligaID

14

In [13]:
player_url

'/player/167495'

In [15]:
teamID

'21158278'

In [19]:
attr_dict

{'Acceleration': '79',
 'Aggression': '37',
 'Agility': '76',
 'Balance': '85',
 'Ball control': '66',
 'Composure': '49',
 'Crossing': '63',
 'Curve': '59',
 'Dribbling': '65',
 'Finishing': '65',
 'Free kick accuracy': '61',
 'GK diving': '6',
 'GK handling': '11',
 'GK kicking': '16',
 'GK positioning': '8',
 'GK reflexes': '15',
 'Heading accuracy': '49',
 'Interceptions': '27',
 'Jumping': '53',
 'Long passing': '55',
 'Long shots': '69',
 'Marking': '32',
 'Penalties': '57',
 'Positioning': '59',
 'Reactions': '51',
 'Short passing': '64',
 'Shot power': '66',
 'Sliding tackle': '31',
 'Sprint speed': '68',
 'Stamina': '51',
 'Standing tackle': '37',
 'Strength': '36',
 'Vision': '55',
 'Volleys': '58'}

In [25]:
attr_dict = attr_dict_base

In [23]:
attr_dict.clear()

In [26]:
attr_dict

{}

In [27]:
attr_dict_base


{}

In [7]:
c.execute('ALTER TABLE player ADD COLUMN "jersey" "INTEGER"')

OperationalError: duplicate column name: jersey

(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)


In [112]:
def get_jersey(player):
    url = ''.join(['https://sofifa.com/player/', str(player[1]), patch_urls[player[0]%1000000]])
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    c.execute("UPDATE player SET jersey=(?) WHERE playerID=(?)",(soup.find_all('ul', class_='pl')[1].find_all('li')[3].text.strip().split('\n')[1],player[0]))
    con.commit()


In [106]:
get_jersey((237985158835, 237985))

[('Kevin Danso', 38)]


In [41]:
plyrs = [plyr for plyr in c.execute('SELECT playerID, sofifaPlayerID FROM player')]

In [113]:
for i,p in enumerate(plyrs):
    get_jersey(p)
    print(i,'/',71337, end='\r')

In [100]:
print([x for x in c.execute('SELECT name,jersey FROM player WHERE playerID IS 237985158835')])

[('Kevin Danso', 38)]


In [99]:
c.execute("UPDATE player SET jersey=(?) WHERE playerID=(?)",(38,237985158835))
con.commit()

In [97]:
print([x for x in c.execute('SELECT name,jersey FROM player WHERE jersey = 38')])

[]


In [92]:
c.description

In [109]:
len(plyrs)

71337

In [115]:
((12*60+47)*60+2)/71337

0.6451350631509595